In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import nltk

10788


In [2]:
fileids = reuters.fileids()

NameError: name 'reuters' is not defined

In [62]:
trainids = []
testids = []
for id in fileids:
    tokens = id.split('/')
    if tokens[0] == 'training':
        trainids.append(id)
    else:
        if tokens[0] == 'test':
            testids.append(id)


In [63]:
import string
exclude = set(string.punctuation)
train_words = []
len1words = {}
for id in trainids:
    words = reuters.words(id)
    words_nopunc = []
    for word in words:
        if word in exclude:
            continue
        else:
            word = word.lower()
            words_nopunc.append(word)
    train_words.append(words_nopunc)

In [64]:

print(exclude)

{'?', '>', ']', '/', ':', '=', '!', '(', ',', '.', '{', '}', '&', '+', '_', ';', '*', '[', '-', '~', '$', '<', '\\', '|', ')', '^', '#', '%', '@', '`', '"', "'"}


In [65]:
import nltk
import string 
#tokenize raw strings
exclude = set(string.punctuation)
train_words = []
exclude = set(string.punctuation)
for id in trainids:
    raw = reuters.raw(id)
    words = nltk.word_tokenize(raw)
    words_nopunc = []
    for word in words:
        if word in exclude:
            continue
        else:
            word = word.lower()
            words_nopunc.append(word)
    train_words.append(words_nopunc)


In [66]:
# remove numbers. they can be like 100 1090,200 2.123 etc
# strategy is to remove punctuation and then check if its an integer
import re


def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def remove_numbers(words_list):
    no_numbers_words_list = []
    for words in words_list:
        words_no_number = []
        for word in words:
            # remove punc from string
            # not working
            # word_no_num = word.translate(None, string.punctuation)
            word_no_num = re.sub(r'[^\w\s]','',word)
            if RepresentsInt(word_no_num):
                continue
            else:
                words_no_number.append(word)
        no_numbers_words_list.append(words_no_number)
    return no_numbers_words_list

train_words_no_num = remove_numbers(train_words)


In [67]:
print(train_words_no_num[0][:20])

['bahia', 'cocoa', 'review', 'showers', 'continued', 'throughout', 'the', 'week', 'in', 'the', 'bahia', 'cocoa', 'zone', 'alleviating', 'the', 'drought', 'since', 'early', 'january', 'and']


In [68]:
unique = {}
for words in train_words_no_num:
    for word in words:
        if word in unique:
            unique[word] += 1
        else:
            unique[word] = 1


In [74]:
print(len(unique))


28354


In [76]:
sorted(unique.items(), key=lambda item: item[1], reverse=True)

[('the', 51383),
 ('to', 27306),
 ('of', 27306),
 ('in', 22003),
 ('and', 18957),
 ('said', 18843),
 ('a', 18456),
 ('mln', 13089),
 ('for', 9852),
 ('vs', 9287),
 ('dlrs', 8781),
 ('it', 8290),
 ('pct', 7394),
 ('on', 6597),
 ("'s", 6415),
 ('lt', 6302),
 ('from', 6291),
 ('cts', 5776),
 ('is', 5661),
 ('its', 5563),
 ('that', 5541),
 ('year', 5307),
 ('at', 5200),
 ('by', 5168),
 ('be', 4744),
 ('net', 4561),
 ('with', 4556),
 ('will', 4509),
 ("''", 4349),
 ('``', 4345),
 ('was', 4225),
 ('billion', 4104),
 ('loss', 4001),
 ('he', 3846),
 ('u.s.', 3575),
 ('has', 3480),
 ('would', 3408),
 ('company', 3369),
 ('an', 3317),
 ('as', 3306),
 ('not', 3243),
 ('inc', 2870),
 ('bank', 2743),
 ('but', 2694),
 ('which', 2691),
 ('this', 2644),
 ('shr', 2614),
 ('last', 2393),
 ('are', 2370),
 ('corp', 2346),
 ('have', 2322),
 ('or', 2318),
 ('trade', 2315),
 ('were', 2295),
 ('oil', 2271),
 ('share', 2261),
 ('profit', 2245),
 ('had', 2203),
 ('about', 2075),
 ('market', 2048),
 ('new', 2010

In [72]:
print(reuters.raw(trainids[0]).split('\n'))

['BAHIA COCOA REVIEW', '  Showers continued throughout the week in', '  the Bahia cocoa zone, alleviating the drought since early', '  January and improving prospects for the coming temporao,', '  although normal humidity levels have not been restored,', '  Comissaria Smith said in its weekly review.', '      The dry period means the temporao will be late this year.', '      Arrivals for the week ended February 22 were 155,221 bags', '  of 60 kilos making a cumulative total for the season of 5.93', '  mln against 5.81 at the same stage last year. Again it seems', '  that cocoa delivered earlier on consignment was included in the', '  arrivals figures.', '      Comissaria Smith said there is still some doubt as to how', '  much old crop cocoa is still available as harvesting has', '  practically come to an end. With total Bahia crop estimates', '  around 6.4 mln bags and sales standing at almost 6.2 mln there', '  are a few hundred thousand bags still in the hands of farmers,', '  middl

In [5]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)
print(vocabulary_size)

30075


In [6]:
print(vocabulary[:30])

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'and', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 'exporting', 'nations', 'that', 'row', 'could']


In [7]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [8]:
len(idx_pairs)

6309176

In [9]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [ ]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 10
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.item()
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:
        print('epo =' + str(epo), 'loss = ' + str(loss_val/len(idx_pairs)))
#         print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')